<a href="https://colab.research.google.com/github/sagihaider/EEG_Deep/blob/master/main_Single_Trial_Classification_NSL_2Class_Grid_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!git clone https://github.com/sagihaider/EEG_Deep.git

fatal: destination path 'EEG_Deep' already exists and is not an empty directory.


In [11]:
import scipy.io as spio
from google.colab import files
import numpy as np
import pandas as pd
import time
from importlib.machinery import SourceFileLoader

# EEGNet-specific imports
from EEG_Deep.EEGModels import EEGNet, ShallowConvNet, DeepConvNet, EEGNet_Patient
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
K.set_image_data_format('channels_first')

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

# tools for plotting confusion matrices
from matplotlib import pyplot as plt
from scipy.signal import butter, lfilter

!pip install neural_structured_learning
import tensorflow as tf
import neural_structured_learning as nsl

In [0]:
# Band-pass Filter
def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data)
    return y


In [15]:
from numpy import zeros
K.clear_session()

# Initialize parameter
h_cut = [24]
drop_out = [0.25,0.5]
k_len = [128] #[32, 64, 128]
n_epochs = 100

out_f_name = 'acc_NSL_EEGNet' + '_epochs_' + str(n_epochs) + '_filter_' + str(h_cut) +'_EEGNet_NSL_ReniData.npy'
print(out_f_name)

nsub = 14
nfilt = len(h_cut)
ndrop = len(drop_out)
nkl = len(k_len)
loss_score = zeros([nsub, nfilt,ndrop,nkl])
categorical_crossentropy_score = zeros([nsub, nfilt,ndrop,nkl])
categorical_accuracy = zeros([nsub, nfilt,ndrop,nkl])

X_tr = np.empty([100, 15, 4096])
X_ts = np.empty([60, 15, 4096])


for sub_idx, x in enumerate(range(1,2)):
    for h_indx, h in enumerate(h_cut):
        fName = 'EEG_Deep/Data2A/Eopched_S0' + str(x) + 'T.mat'  # Load Data
        print(fName)
        mat = spio.loadmat(fName)
        r_X_tr = mat['cleanRawEEGData']
        y_tr = mat['cleanClassLabels']
        y_tr = y_tr.flatten() 
  
        print(np.shape(r_X_tr))
        print(np.shape(y_tr))

        for t in range(r_X_tr.shape[0]):
          tril = r_X_tr[t,:,:]
          # tril = tril.transpose()
          tril_filtered = butter_bandpass_filter(tril, lowcut=4, highcut=h, fs=250, order=4)
          # tril_filtered = tril_filtered.transpose()
          X_tr[t,:,:] = tril_filtered 
    
        # split data of each subject in training and validation
        X_train      = X_tr[0:80,:,1536:3072]
        Y_train      = y_tr[0:80]
        X_val       = X_tr[80:,:,1536:3072]
        Y_val       = y_tr[80:]

        print(np.shape(X_train))
        print(np.shape(Y_train))
        print(np.shape(X_val))
        print(np.shape(Y_val))
  
        # convert labels to one-hot encodings.
        Y_train      = np_utils.to_categorical(Y_train-1)
        Y_val       = np_utils.to_categorical(Y_val-1)

        kernels, chans, samples = 1, 15, 1536
        # convert data to NCHW (trials, kernels, channels, samples) format. Data 
        # contains 22 channels and 500 time-points. Set the number of kernels to 1.
        X_train      = X_train.reshape(X_train.shape[0], kernels, chans, samples)
        X_val       = X_val.reshape(X_val.shape[0], kernels, chans, samples)
   
        print('X_train shape:', X_train.shape)
        print(X_train.shape[0], 'train samples')
        print(X_val.shape[0], 'val samples')

        fName = 'EEG_Deep/Data2A/Eopched_S0' + str(x) + 'E.mat'  # Load Data
        print(fName)
        mat = spio.loadmat(fName)
        r_X_ts = mat['cleanRawEEGData']
        y_ts = mat['cleanClassLabels']
        y_ts = y_ts.flatten() 

        print(np.shape(r_X_ts))
        print(np.shape(y_ts))

        for t in range(r_X_ts.shape[0]):
          tril = r_X_ts[t,:,:]
          # tril = tril.transpose()
          tril_filtered = butter_bandpass_filter(tril, lowcut=4, highcut=h, fs=250, order=4)
          # tril_filtered = tril_filtered.transpose()
          X_ts[t,:,:] = tril_filtered 

        X_test      = X_ts[:,:,1536:3072]
        Y_test      = y_ts[:]
        print(np.shape(X_test))
        print(np.shape(Y_test))

        #convert labels to one-hot encodings.
        Y_test      = np_utils.to_categorical(Y_test-1)

        # convert data to NCHW (trials, kernels, channels, samples) format. Data 
        # contains 22 channels and 500 time-points. Set the number of kernels to 1.
        X_test      = X_test.reshape(X_test.shape[0], kernels, chans, samples)

        print('X_train shape:', X_test.shape)
        print(X_test.shape[0], 'test samples')

        for id_d, d in enumerate(drop_out):
            for id_kl, kl in enumerate(k_len):
                print(id_kl, id_d)
                # configure the EEGNet-8,2,16 model with kernel length of 32 samples (other 
                # model configurations may do better, but this is a good starting point)
                model = EEGNet(nb_classes = 2, Chans = 15, Samples = 1536, dropoutRate = d, 
                                kernLength = kl, F1 = 8, D = 2, F2 = 16, norm_rate = 0.25, 
                                dropoutType = 'Dropout')

                adv_config = nsl.configs.make_adv_reg_config(multiplier=0.2, adv_step_size=0.5, adv_grad_norm='infinity')
                adv_model = nsl.keras.AdversarialRegularization(model, adv_config=adv_config)
  
                # compile the model and set the optimizers
                adv_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
                batch_size = 32

                X_train = tf.cast(X_train, tf.float32)
                X_test = tf.cast(X_test, tf.float32)
                X_val = tf.cast(X_val, tf.float32)

                train_data = tf.data.Dataset.from_tensor_slices({'input': X_train, 'label': Y_train}).batch(batch_size)
                val_data = tf.data.Dataset.from_tensor_slices({'input': X_val, 'label': Y_val}).batch(batch_size)
                test_data = tf.data.Dataset.from_tensor_slices({'input': X_test, 'label': Y_test}).batch(batch_size)

                val_steps = X_val.shape[0] // batch_size
  
                history = adv_model.fit(train_data, validation_data=val_data, validation_steps=None, 
                                        epochs=n_epochs, verbose=1)

                print('\n# Evaluate on test data')
                results = adv_model.evaluate(test_data)
                print('test loss, test acc:', results)

                loss_score[sub_idx,h_indx,id_d, id_kl] = results[0]
                categorical_crossentropy_score[sub_idx,h_indx,id_d, id_kl] = results[1]
                categorical_accuracy[sub_idx,h_indx,id_d, id_kl] = results[2]

                from keras import backend as K 
                K.clear_session()

np.save(out_f_name, loss_score, categorical_crossentropy_score, categorical_accuracy)


acc_NSL_EEGNet_epochs_100_filter_[24]_EEGNet_NSL_ReniData.npy
EEG_Deep/Data2A/Eopched_S01T.mat
(100, 15, 4096)
(100,)
(80, 15, 1536)
(80,)
(20, 15, 1536)
(20,)
X_train shape: (80, 1, 15, 1536)
80 train samples
20 val samples
EEG_Deep/Data2A/Eopched_S01E.mat
(60, 15, 4096)
(60,)
(60, 15, 1536)
(60,)
X_train shape: (60, 1, 15, 1536)
60 test samples
